In [6]:
# 9.2 이미지 분할 예제
# 필요한 라이브러리 설치
#!pip install tensorflow wget

# 데이터 다운로드
import wget
import tarfile

url = 'https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz'
wget.download(url)
url = 'https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz'
wget.download(url)

with tarfile.open('images.tar.gz') as tar:
    tar.extractall()

with tarfile.open('annotations.tar.gz') as tar:
    tar.extractall()

# 데이터 전처리
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

input_dir = 'images/'
target_dir = 'annotations/trimaps/'
img_size = (160, 160)
num_imgs = len(os.listdir(input_dir))

input_img_paths = sorted([
    os.path.join(input_dir, fname)
    for fname in os.listdir(input_dir)
    if fname.endswith('.jpg')
])
target_img_paths = sorted([
    os.path.join(target_dir, fname)
    for fname in os.listdir(target_dir)
    if fname.endswith('.png') and not fname.startswith('.')
])

# 리스트 길이 출력
print(f"Number of input images: {len(input_img_paths)}")
print(f"Number of target images: {len(target_img_paths)}")

# 리스트의 몇 개 항목 출력
print(f"Sample input image paths: {input_img_paths[:5]}")
print(f"Sample target image paths: {target_img_paths[:5]}")

def load_image(path, size):
    return img_to_array(load_img(path, target_size=size)) / 255.

def load_mask(path, size):
    img = img_to_array(load_img(path, target_size=size, color_mode="grayscale"))
    img[img == 1.0] = 0
    img[img == 2.0] = 1
    img[img == 3.0] = 2
    return img

# num_imgs 값을 input_img_paths와 target_img_paths의 길이로 설정
num_imgs = min(len(input_img_paths), len(target_img_paths))

x = np.zeros((num_imgs, img_size[0], img_size[1], 3), dtype="float32")
y = np.zeros((num_imgs, img_size[0], img_size[1], 1), dtype="uint8")

for i in range(num_imgs):
    x[i] = load_image(input_img_paths[i], img_size)
    y[i] = load_mask(target_img_paths[i], img_size)

# 데이터셋 나누기
val_samples = 1000
train_x, val_x = x[:-val_samples], x[-val_samples:]
train_y, val_y = y[:-val_samples], y[-val_samples:]

# 모델 정의
from tensorflow.keras import layers

def get_model(img_size, num_classes):
    inputs = tf.keras.Input(shape=img_size + (3,))
    x = layers.Conv2D(32, 3, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2DTranspose(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2DTranspose(32, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(num_classes, 1, activation="softmax")(x)
    model = tf.keras.Model(inputs, x)
    return model

# 모델 컴파일 및 훈련
model = get_model(img_size, 3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("oxford_segmentation.h5", save_best_only=True)
]

history = model.fit(train_x, train_y, epochs=50, callbacks=callbacks, validation_data=(val_x, val_y))


Number of input images: 7390
Number of target images: 7390
Sample input image paths: ['images/Abyssinian_1.jpg', 'images/Abyssinian_10.jpg', 'images/Abyssinian_100.jpg', 'images/Abyssinian_101.jpg', 'images/Abyssinian_102.jpg']
Sample target image paths: ['annotations/trimaps/Abyssinian_1.png', 'annotations/trimaps/Abyssinian_10.png', 'annotations/trimaps/Abyssinian_100.png', 'annotations/trimaps/Abyssinian_101.png', 'annotations/trimaps/Abyssinian_102.png']
Epoch 1/50


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1155, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1249, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 620, in update_state
        metric_obj

In [ ]:
# 9-3. 아키텍처
# 필요한 라이브러리 설치
# !pip install tensorflow

# 모델 정의
import tensorflow as tf
from tensorflow.keras import layers, models

def residual_block(x, filters, kernel_size=3):
    shortcut = x
    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.add([shortcut, x])
    x = layers.Activation('relu')(x)
    return x

input_shape = (128, 128, 3)
num_classes = 10

inputs = tf.keras.Input(shape=input_shape)
x = layers.Conv2D(64, 3, padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

x = residual_block(x, 64)
x = residual_block(x, 64)

x = layers.MaxPooling2D()(x)
x = residual_block(x, 128)
x = residual_block(x, 128)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs, x)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

# 가상 데이터로 훈련 (실제 데이터로 교체 가능)
x_train = np.random.rand(100, 128, 128, 3)
y_train = np.random.randint(0, 10, 100)

history = model.fit(x_train, y_train, epochs=10)


In [ ]:
# 9.4 컨브넷이 학습한 것 해석하기
# 필요한 라이브러리 설치
# !pip install tensorflow

# 모델 정의 및 학습 (간단한 예제)
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
import matplotlib.pyplot as plt

model = VGG16(weights='imagenet')

# 이미지 로드 및 전처리
img_path = 'elephant.jpg'  # 여기에 이미지 파일 경로를 입력하세요.
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# 예측
preds = model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

# CAM 기법
class_output = model.output[:, np.argmax(preds[0])]
last_conv_layer = model.get_layer('block5_conv3')

grads = tf.keras.backend.gradients(class_output, last_conv_layer.output)[0]
pooled_grads = tf.keras.backend.mean(grads, axis=(0, 1, 2))
iterate = tf.keras.backend.function([model.input], [pooled_grads, last_conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value = iterate([x])

for i in range(pooled_grads_value.shape[-1]):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

heatmap = np.mean(conv_layer_output_value, axis=-1)

# Heatmap 시각화
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)
plt.show()

# Heatmap을 원본 이미지에 덧붙이기
import cv2

img = cv2.imread(img_path)
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

superimposed_img = heatmap * 0.4 + img
cv2.imwrite('elephant_cam.jpg', superimposed_img)
